In [42]:
from __future__ import division

# =============================================================================
# IMPORTS
# =============================================================================

import os
import pdb
import sys 

import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import numpy as np

from spaceKLIP import database, coron1pipeline, coron2pipeline, coron3pipeline, pyklippipeline, imagetools, analysistools

# plotting
import matplotlib.pyplot as plt

In [49]:
# Set the input and output directories and grab the input FITS files.
reduced = False
cleanalign = False
aligned = False

blur = True
pad = True
coadd = False
crop = False

data_root='DATA/JWST/NIRCam/01448/F480M/'

if aligned and coadd:
    input_dir = os.path.join(data_root, 'coadded/')
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif aligned:
    # input_dir = './spaceklip/aligned/'
    input_dir = os.path.join(data_root, 'aligned/')#f'./spaceklip/aligned{epoch}/'
    # input_dir = './spaceklip/aligned/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleanalign:
    input_dir = os.path.join(data_root, 'recentered/')
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif reduced:
    input_dir = os.path.join(data_root, 'stage2/')
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])
else:
    # input_dir = './04558/'
    input_dir = os.path.join(data_root, 'uncal/') #f'/Users/wbalmer/data/jwst/aflep/uncal{epoch}/'
    # input_dir = '/Users/wbalmer/data/jwst/aflep/uncal_e3/'
    # input_dir = './uncal_e2/'
    # input_dir = './uncal_e3/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])

# additional PSF references dir
ref_ids = []
additionalrefs = []
for i in range(len(ref_ids)):
    input_dir_ref = 'DATA/JWST/NIRCam/%s/F480M/uncal/' % ref_ids[i]
    # read files in additional PSF references input dir
    additionalrefs = additionalrefs + sorted([input_dir_ref + f for f in os.listdir(input_dir_ref) if f.endswith('.fits')])
# list of PSF reference files in main input dir
#psffitsfiles = [f for f in fitsfiles if 'jw04558003001' in f]+[f for f in fitsfiles if 'jw04558006001' in f]+[f for f in fitsfiles if 'jw04558009001' in f]
psffitsfiles = [f for f in fitsfiles if '10001' in f]
# global list of ALL files
allpaths = fitsfiles + additionalrefs
# global list of all PSF reference files 
psfpaths = additionalrefs + psffitsfiles


output_dir = data_root

In [50]:
# Initialize the spaceKLIP database and read the input FITS files.
Database = database.Database(output_dir=output_dir)
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=psfpaths,
                             bgpaths=None)

[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_NIRCAM_NRCBLONG_F480M_CLEAR_NONE_FULL
 TYPE   EXP_TYPE DATAMODL TELESCOP ... BUNIT      ROLL_REF      BLURFWHM
------ --------- -------- -------- ... ----- ------------------ --------
   SCI NRC_IMAGE   STAGE0     JWST ...    DN 273.21303545364003      nan
   SCI NRC_IMAGE   STAGE0     JWST ...    DN   273.213005067784      nan
   SCI NRC_IMAGE   STAGE0     JWST ...    DN  273.2132490364184      nan
SCI_BG NRC_IMAGE   STAGE0     JWST ...    DN  229.6612460254641      nan
SCI_BG NRC_IMAGE   STAGE0     JWST ...    DN 229.66138104316786      nan
SCI_BG NRC_IMAGE   STAGE0     JWST ...    DN  229.6611882703116      nan
   SCI NRC_IMAGE   STAGE0     JWST ...    DN 172.72211377042873      nan
   SCI NRC_IMAGE   STAGE0     JWST ...    DN 172.72195238194033      nan
   SCI NRC_IMAGE   STAGE0     JWST ...    DN 172.72221068076723      nan
SCI_BG NRC_IMAGE   STAGE0     JWST ...    DN 

In [51]:
Database.summarize()

NIRCAM_F480M_NONE
	STAGE0: 24 files;	15 SCI, 0 REF, 9 BG


In [52]:
if not reduced:
    coron1pipeline.run_obs(database=Database,
                           steps={'saturation': {'n_pix_grow_sat': 1,
                                                 'grow_diagonal': False},
                                  'refpix': {'odd_even_columns': True,
                                             'odd_even_rows': True,
                                             'nlower': 4,
                                             'nupper': 4,
                                             'nleft': 4,
                                             'nright': 4,
                                             'nrow_off': 0,
                                             'ncol_off': 0},
                                  'dark_current': {'skip': True},
                                  'persistence': {'skip': True},
                                  'jump': {'rejection_threshold': 4.,
                                           'three_group_rejection_threshold': 4.,
                                           'four_group_rejection_threshold': 4.,
                                           'maximum_cores': 'all'},
                                  'ramp_fit': {'save_calibrated_ramp': False,
                                               'maximum_cores': 'all'}},
                            subdir='stage1')

if not reduced:
    coron2pipeline.run_obs(database=Database,
                           steps={'outlier_detection': {'skip': False}},
                           subdir='stage2')

[spaceKLIP.coron1pipeline:INFO] --> Concatenation JWST_NIRCAM_NRCBLONG_F480M_CLEAR_NONE_FULL
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448001001_08101_00001_nrcblong_uncal.fits
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448001001_08101_00002_nrcblong_uncal.fits
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448001001_08101_00003_nrcblong_uncal.fits
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448002001_08101_00001_nrcblong_uncal.fits
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448002001_08101_00002_nrcblong_uncal.fits
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448002001_08101_00003_nrcblong_uncal.fits
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448003001_08101_00001_nrcblong_uncal.fits
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01448003001_08101_00002_nrcblong_uncal.fits
[spaceKLIP.coron1pi

KeyboardInterrupt: 